In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
from matplotlib import colormaps
import numpy as np
from sklearn.metrics import accuracy_score
from utils import transforms, exports

split_path = Path("results/split")

In [ ]:
with exports.JSONExporter(Path("analysis") / "figs" / "train-sizes.json") as data:
    dim_splits = data["train_sizes"]

In [ ]:
values = np.load(
    split_path / "results_split_5_mamba_mw_1.7.npy",
    allow_pickle=True,
).item()
terrains = np.array([t for t in values["terrains"] if t != "MIXED"])
inv_terrains = {t: i for i, t in enumerate(terrains)}

In [ ]:
res_cnn = []
res_mamba = []
for p in sorted(split_path.iterdir()):
    if "CNN" in p.stem:
        res_cnn.append(np.load(p, allow_pickle=True).item())
    elif "mamba" in p.stem:
        res_mamba.append(np.load(p, allow_pickle=True).item())

In [ ]:
cnn_acc_per_split = []
mamba_acc_per_split = []
for res in res_cnn[::-1]:
    ypred = transforms.convert_to_idx(res["pred"], inv_terrains)
    ytest = transforms.convert_to_idx(res["true"], inv_terrains)
    fold_size = len(ypred) // 5
    print(fold_size)
    acc = []
    for k in range(5):
        pred = ypred[k * fold_size : (k + 1) * fold_size]
        true = ytest[k * fold_size : (k + 1) * fold_size]
        acc.append(accuracy_score(true, pred))
    cnn_acc_per_split.append(acc)

for res in res_mamba[::-1]:
    ypred = transforms.convert_to_idx(res["pred"], inv_terrains)
    ytest = transforms.convert_to_idx(res["true"], inv_terrains)
    fold_size = len(ypred) // 5
    print(fold_size)
    acc = []
    for k in range(5):
        pred = ypred[k * fold_size : (k + 1) * fold_size]
        true = ytest[k * fold_size : (k + 1) * fold_size]
        acc.append(accuracy_score(true, pred))
    mamba_acc_per_split.append(acc)


print(f"{cnn_acc_per_split=}")
print(f"{mamba_acc_per_split=}")

In [ ]:
x = np.array(dim_splits)[::-1]

cnn_acc = np.mean(cnn_acc_per_split, axis=1)
mamba_acc = np.mean(mamba_acc_per_split, axis=1)

cnn_err = 1 - cnn_acc
mamba_err = 1 - mamba_acc

cnn_bars = 1 - np.percentile(cnn_acc_per_split, [75, 25], axis=1).T
mamba_bars = 1 - np.percentile(mamba_acc_per_split, [75, 25], axis=1).T

COLORS = {
    "CNN": "mediumvioletred",
    "MAMBA": "teal",
}

In [ ]:
with plt.style.context("analysis/figure.mplstyle"):
    plt.rcParams.update(
        {
            "font.size": 7,
            "axes.titlesize": 7,
            "axes.labelsize": 16,
            "xtick.labelsize": 16,
            "ytick.labelsize": 16,
        }
    )

    fig, ax = plt.subplots(figsize=(8, 4.8))

    ax.scatter(x, cnn_err, label="CNN", color=COLORS["CNN"])
    ax.scatter(x, mamba_err, label="Mamba", color=COLORS["MAMBA"])

    ax.fill_between(x, cnn_bars[:, 0], cnn_bars[:, 1], alpha=0.1, color=COLORS["CNN"])
    ax.fill_between(
        x, mamba_bars[:, 0], mamba_bars[:, 1], alpha=0.1, color=COLORS["MAMBA"]
    )

    print(fig.get_size_inches())

    xlog = np.log(x)

    # cnn_trend = np.poly1d(np.polyfit(np.log2(x), cnn_err, 2))
    # mamba_trend = np.poly1d(np.polyfit(np.log2(x), mamba_err, 2))
    cnn_trend = np.poly1d(np.polyfit(xlog, np.log(cnn_err), 1))
    mamba_trend = np.poly1d(np.polyfit(xlog, np.log(mamba_err), 1))

    # x_trend
    ratio = x[-1] / x[-2]
    x_extend = x[-1] * ratio ** np.arange(2)
    x_trend = np.hstack([x[:-1], x_extend])
    # x_trend = x

    cnn_fit = lambda x: np.exp(cnn_trend(np.log(x)))
    mamba_fit = lambda x: np.exp(mamba_trend(np.log(x)))

    ax.loglog(x_trend, cnn_fit(x_trend), linestyle="--", color=COLORS["CNN"])
    ax.loglog(x_trend, mamba_fit(x_trend), linestyle="--", color=COLORS["MAMBA"])

    ax.set_xlabel("Train dataset size (samples)")
    ax.set_ylabel("Test error ($\%$)")

    ax.yaxis.set_minor_formatter(mticker.NullFormatter())

    ax.set_yticks(np.arange(0.1, 0.5, 0.1))
    ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f"{y:.0%}"))
    # ax.yaxis.set_major_formatter('{x:g}')
    # ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
    # ax.yaxis.get_major_formatter().set_scientific(False)
    # ax.ticklabel_format(axis="y", style="plain")
    # ax.yaxis.set_major_formatter(mticker.FuncFormatter(lambda y, _: f"{y*100:%}"))

    # ax.yaxis.set_major_formatter(mticker.ScalarFormatter())
    # ax.ticklabel_format(axis="y", style="plain")

    ax.legend(loc="best")
    # fig.tight_layout()
    fig.savefig("analysis/figs/train-sizes.pdf", bbox_inches="tight")

plt.show()